# Sistemas Inteligentes 2020/2021

## Mini-projeto 2: Puzzle dos quadrados de fósforos - heurísticas

## Relatório

<img src="fosforos.gif" alt="Drawing" style="width: 100px;"/>

#### Grupo: 32

Número: 53749 - Nome: Pedro Rodrigues

Número: 53354 - Nome: João Viana

Número: 53745 - Nome: João Nunes

In [1]:
from copy import copy
import random
import timeit
from search_v3 import *

# quadrados possiveis de lado 1, 2, 3
quad_side_1 = [set([1,4,5,8]), set([2,5,6,9]), set([3,6,7,10]), 
     set([8,11,12,15]), set([9,12,13,16]), set([10,13,14,17]), 
     set([15,18,19,22]), set([16,19,20,23]), set([17,20,21,24])]

quad_side_2 = [set([1,2,4,6,11,13,15,16]), 
     set([2,3,5,7,12,14,16,17]),
     set([8,9,11,13,18,20,22,23]),
     set([9,10,12,14,19,21,23,24])]

quad_side_3 = set([1,2,3,4,7,11,14,18,21,22,23,24])

class ProblemaFosforos(Problem):
    quadrados = [quad_side_3]
    quadrados.extend(quad_side_2)
    quadrados.extend(quad_side_1)

    #hor = [1,2,3,8,9,10,15,16,17,22,23,24]
    #ver = [4,5,6,7,11,12,13,14,18,19,20,21]
    def __init__(self, initial, goal=2):
        self.initial = tuple(initial)
        self.goal = goal

    def actions(self, estado):
        """ Cada acção representa a remoção de um fósforo. Existem 24 acções no total.
         Por exemplo, a acção 1 corresponde a remover o fósforo 1.

         Na situação em que não há nenhum quadrado formado, não adianta realizar mais ações -> []

        Args:
            estado ([type]): [description]

        Returns:
            [type]: [description]
        """

        if len(self.quadrados_validos(estado)) == 0:
            return []
        return list(estado)

    def result(self, estado, accao):
        """Remove o fósforo com id=acçao do estado e retorna um novo estado.

        Args:
            estado (set[int]): conjunto de IDs de fósforos
            accao (int): ID do fósforo a remover

        Returns:
            tuple[int]: novo estado
        """
        s = copy(list(estado))
        s.remove(accao)
        return tuple(s)

    def goal_test(self, estado):
        """Testa se o estado é o estado final.
        O objetivo é ter apenas self.goal quadrados válidos e nenhum fósforo solto.

        Args:
            estado (tuple[int]): estado a ser avaliado.

        Returns:
            bool: se é o estado final.
        """
        qval = self.quadrados_validos(estado)
        soltos = self.fosforos_soltos(estado)
        return len(qval) == self.goal and soltos == 0

    def quadrados_validos(self, estado):
        """Verifica quantos são quadrados válidos estão presentes no estado, usando a informação estática da classe.

        Args:
            estado (tuple[int]): estado a ser avaliado.

        Returns:
            list: lista com os IDs dos fósforos de quadrados válidos
        """
        qval = []
        for i in self.quadrados:
            if i.intersection(set(estado)) == i:
                qval.append(list(i))
        return qval

    def fosforos_soltos(self, estado):
        """Devolve o número de fósforos soltos do estado.

        Args:
            estado (tuple[int]): estado a ser avaliado.

        Returns:
            int: numero de fósforos soltos.
        """
        qval = self.quadrados_validos(estado)
        qval = set([item for sublist in qval for item in sublist])
        return len(set(estado) - qval)

    def display(self, estado):
        """Imprime uma visualização do estado

        Args:
            estado (tuple[int]): estado a ser avaliado.
        """
        N = 3
        for i in range(1, N + 2):
            h = [2 * (i - 1) * N + i + j
                 for j in range(0, N)]  # fosforos horizontais
            v = [(2 * i - 1) * N + i + j
                 for j in range(0, N + 1)]  # fosforos verticais
            for k in h:
                if k in estado:
                    print('  -- ', end='')
                else:
                    print('     ', end='')
            print()
            for k in v:
                if k in estado:
                    print('|    ', end='')
                else:
                    print('     ', end='')
            print()
            
    def h1(self, no):
        """ heurística a ser usada pelos algoritmos de procura informada.
        Args:
            no (Node): objeto da classe Node, definida na search_v3.py
        Returns:
            number: valor_da_heuristica do nó 
        """
        if self.fosforos_soltos(no.state) > 8:
            return len(self.initial)
        else:
            return 24-(self.fosforos_soltos(no.state))
    
    def h2(self, no):
        """ heurística a ser usada pelos algoritmos de procura informada.
        Args:
            no (Node): objeto da classe Node, definida na search_v3.py
        Returns:
            number: valor_da_heuristica do nó 
        """
        
        return len(self.quadrados_validos(no.state))

## Parte 1

1. Considerando a formulação descrita anteriomente e utilizando o princípio da contagem fundamental, o tamanho do espaço de estados é 24!.


2. Visto que os custos são homogéneos esta solução é óptima e encontra-se a uma profundidade de 8.


3. Uma Heurística é definida por uma função que estima a proximidade de um estado ao objetivo. Assim, na heurística h1, para cumprirmos a condição imposta de obter 2 quadrados, concluímos que têm de existir 8 fósforos soltos e aplicamos uma condição que se o número de fósforos soltos fosse superior a 8 não seria possível formar 2 quadrados nesse estado. Para o caso de o número de fosforos soltos ser inferior ou igual a 8 e quantos mais fósforos soltos existirem nesse intervalo entre 0 e 8, menor seria o custo, já que quanto maior for o número de fósforos soltos nesse intervalo mais próximos estaremos do estado final.
Já na heurística 2, quanto menor for o número de quadrados válidos mais próximos estaremos do objetivo final e menor será o custo da solução.


4. Para que uma heurística seja admíssivel ```h(x) <= h*(x)```.

   Na função 1 ```h1(x) = 24 e h*(x) = 8```, logo, não é admissível.

   Na função 2 ```h2(x) = 9 e h*(x) = 8```, logo, também não é admissível.

   Portanto, podemos concluir que ambas as heurísticas não são admissíveis.

5. Algoritmo ```aprofundamento_prog_astar```

In [2]:
def aprofundamento_prog_astar(problem, f, display=False, max_depth=50):
    """
    problem: o problema
    h: a heurística
    max_depth: a profundidade máxima permitida
    """
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue("min", f)
    frontier.append(node)
    explored = set()
    inc = 0
    while frontier and x <= max_depth:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and",
                      len(frontier), "paths remain in the frontier")
            return (node)
        else:
            inc += 1
            if inc > max_depth:
                max_depth += 1
            explored.add(node.state)
            for child in node.expand(problem):
                if child.state not in explored and child not in frontier:
                    frontier.append(child)
                elif child in frontier:
                    if f(child) < frontier[child]:
                        del frontier[child]
                        frontier.append(child)
    return None
    
    


6 - Construa a [tabela](https://docs.github.com/en/github/writing-on-github/organizing-information-with-tables) pedida.

|Método|Tempo|Nós Expandidos|Nós visitados|Custo da Solução|
|:-|---|---|---|---|
|Largura-primeiro|54.51001119999998|-|-|8|
|Custo-uniforme|104.03733549999998|598788|1050995|8|
|Greedy H1|1056.6893169999998|4440817|7820341|9|
|Greedy H2|69.23725100000001|1154564|6227|16|
|A* H1|202.7043716|1218414|917729|8|
|A* H2|294.5390778000001|1218414|917729|8|
|Aprofundamento progressivo A* H1|767.250293|7820341|4440817|9|
|Aprofundamento progressivo A* H2|67.01035540000004|1154564|6227|16|


### Código correspondente aos resultados da tabela.

In [3]:
x = [i for i in range(1,25)]
p1 = ProblemaFosforos(initial = x)

##Procura em largura-primeiro##

##start = timeit.default_timer()
##res_breadth = breadth_first_graph_search(p1)
##stop = timeit.default_timer()
##timeGraph = stop - start
##print("Time:", timeGraph)
##print("Custo:", str(res_breadth.path_cost))

##Procura de custo uniforme##

##start = timeit.default_timer()
##res_uniform = uniform_cost_search(p1,display=True)
##stop = timeit.default_timer()
##timeGraph = stop - start
##print("Time:", timeGraph)
##print("Custo:", str(res_uniform.path_cost))


##Greedy com h1##

##start = timeit.default_timer()
##res_greedy = best_first_graph_search(p1,p1.h1,display=True)
##stop = timeit.default_timer()
##timeGraph = stop - start
##print("Time:", timeGraph)
##print("Custo:", str(res_greedy.path_cost))

##Greedy com h2##

#start = timeit.default_timer()
#res_greedy = best_first_graph_search(p1,p1.h2,display=True)
#stop = timeit.default_timer()
#timeGraph = stop - start
#print("Time:", timeGraph)
#print("Custo:", str(res_greedy.path_cost))


##Algoritmo A* com h1###

#start = timeit.default_timer()
#res_astar = astar_search(p1,p1.h2,display=True)
#stop = timeit.default_timer()
#timeGraph = stop - start
#print("Time:", timeGraph)
#print("Custo:", str(res_astar.path_cost))

##Algoritmo A* com h2###

#start = timeit.default_timer()
#res_astar = astar_search(p1,p1.h2,display=True)
#stop = timeit.default_timer()
#timeGraph = stop - start
#print("Time:", timeGraph)
#print("Custo:", str(res_astar.path_cost))


##Aprofundamento progressivo com A* para h1##

#start = timeit.default_timer()
#res_prof_astar = aprofundamento_prog_astar(p1,p1.h1,display=True, max_depth = 50)
#stop = timeit.default_timer()
#timeGraph = stop - start
#print("Time:", timeGraph)
#print("Custo:", str(res_prof_astar.path_cost))



##Aprofundamento progressivo com A* para h2##

#start = timeit.default_timer()
#res_prof_astar = aprofundamento_prog_astar(p1,p1.h2,display=True)
#stop = timeit.default_timer()
#timeGraph = stop - start
#print("Time:", timeGraph)
#print("Custo:", str(res_prof_astar.path_cost))

